In [ ]:
from roboflow import Roboflow
import supervision as sv
import cv2
import numpy as np
import os

# Initialize Roboflow
rf = Roboflow(api_key="zFMAHTN2KKDonwd6JydE")
project = rf.workspace().project("carplate-xuk6s")
model = project.version(1).model

# Path to the directory containing images
image_folder = "../images"

# List all image files in the directory
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

# Iterate over each image file
for image_file in image_files:
    # Construct the full path to the image
    image_path = os.path.join(image_folder, image_file)
    
    # Load the image
    image = cv2.imread(image_path)
    
    # Perform detection
    result = model.predict(image_path, confidence=40, overlap=30).json()
    labels = [item["class"] for item in result["predictions"]]
    
    # Convert predictions to the required format
    xyxy = np.array([
        (
            item["x"] - item["width"] / 2,
            item["y"] - item["height"] / 2,
            item["x"] + item["width"] / 2,
            item["y"] + item["height"] / 2
        )
        for item in result["predictions"]
    ])
    
    detections = sv.Detections(
        xyxy=xyxy,
        confidence=np.array([item["confidence"] for item in result["predictions"]]),
        class_id=np.array([item["class_id"] for item in result["predictions"]])
    )
    
    # Initialize annotators
    label_annotator = sv.LabelAnnotator()
    bounding_box_annotator = sv.BoxAnnotator()
    
    # Annotate the image
    annotated_image = bounding_box_annotator.annotate(scene=image, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
    
    # Display the annotated image
    sv.plot_image(image=annotated_image, size=(16, 16))
    # Optionally save the annotated image
    # cv2.imwrite(f"annotated_{image_file}", annotated_image)

In [ ]:
import cv2
import pytesseract
import re
import os

# Define the folder containing the images
folder_path = "plates"

# Tesseract configuration for optimal performance
custom_config = r'--oem 3 --psm 6'

# Define a function to clean up common OCR errors
def clean_text(text):
    # Remove trailing or isolated characters (like "ន" at the end)
    cleaned = re.sub(r'\s+[ន]*$', '', text.strip())
    return cleaned

# Iterate through all images in the "plates" folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Process only image files
        image_path = os.path.join(folder_path, filename)

        # Load and preprocess the image
        image = cv2.imread(image_path)

        # Resize the image for better recognition
        image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)

        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(image, (5, 5), 0)

        # Convert to grayscale and apply adaptive thresholding
        gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
        )

        # Perform OCR on the preprocessed image
        raw_text = pytesseract.image_to_string(thresh, config=custom_config, lang='khm+eng')

        # Apply the clean_text function to each line of the OCR result
        lines = raw_text.strip().split('\n')
        cleaned_lines = [clean_text(line) for line in lines]

        # Display the cleaned OCR result
        print(f"\n[Cleaned Text from {filename}]:")
        for line in cleaned_lines:
            print(line)
